## Homework II for ISDN6380B

Siyan HU, student no. 12217679

### 1. Load Dataset and Generate Labels

In [1]:
import sys
import os
import random

cwd = os.getcwd()
sys.path.append(cwd)

import file_io as fio

Load dataset

In [5]:
full_image_folder = fio.createPath(fio.sep, [cwd, 'data', 'images'])
full_image_paths = fio.traverse_dir(full_image_folder, full_path=True, towards_sub=False)
full_image_paths = fio.filter_ext(filepath_list=full_image_paths, filter_out_target=False, ext_set=fio.img_ext_set)

full_bbox_folder = fio.createPath(fio.sep, [cwd, 'data', 'labels'])
full_bbox_paths = fio.traverse_dir(full_bbox_folder, full_path=True, towards_sub=False)
full_bbox_paths = fio.filter_ext(filepath_list=full_bbox_paths, filter_out_target=False, ext_set=['txt'])

if (len(full_image_paths) == len(full_bbox_paths)) == False:
    print("ERROR: check files again or add a matching function.")

In [7]:
trainval_base = 1.0
trainval_lod = 0.9
train_lod = 0.9

total_sample_num = len(full_image_paths)
list_index = range(total_sample_num)

tv_num = round(total_sample_num * trainval_lod)
tr_num = round(tv_num * train_lod)

trainval_set = random.sample(list_index, tv_num)
train_set = random.sample(trainval_set , tr_num)
val_set = list(set(trainval_set).difference(set(train_set)))
test_set = list(set(list_index).difference(set(trainval_set)))

split_dir = fio.createPath(fio.sep, [cwd, 'data', 'dataset'])

for i in list_index:
    img_path = full_image_paths[i]
    if i in trainval_set :
        with open(split_dir + fio.sep + 'trainval.txt', 'a+') as file_trainval:
            file_trainval.write(img_path + '\n')
        if i in train_set :
            with open(split_dir + fio.sep + 'train.txt', 'a+') as file_train:
                file_train.write(img_path + '\n')
        else:
            with open(split_dir + fio.sep + 'val.txt', 'a+') as file_val:
                file_val.write(img_path + '\n')
    else:
        with open(split_dir + fio.sep + 'test.txt', 'a+') as file_test:
            file_test.write(img_path + '\n')

print("Spliting database: {} train+validation samples.\n {} are pure training data, \nand {} testing images".format(trainval_set, train_set, test_set))

Spliting database: [19, 24, 32, 3, 18, 41, 8, 7, 16, 0, 25, 21, 17, 36, 38, 9, 40, 50, 37, 44, 15, 20, 14, 47, 31, 33, 4, 12, 6, 46, 13, 48, 11, 39, 1, 49, 27, 43, 5, 45, 2, 34, 10, 28, 35, 26] train+validation samples.
 [3, 35, 12, 36, 46, 49, 39, 14, 16, 17, 44, 41, 6, 1, 47, 37, 18, 45, 40, 27, 10, 50, 19, 5, 0, 11, 2, 9, 4, 31, 32, 24, 20, 43, 15, 34, 38, 25, 26, 48, 28] are pure training data, 
and [42, 22, 23, 29, 30] testing images


create yaml

In [8]:
yaml_filepath = fio.createPath(fio.sep, [cwd, 'data'], 'chessboard.yaml')

In [9]:
with open(yaml_filepath, 'w', encoding='utf-8') as f:
    f.write('train: ' + split_dir + fio.sep + 'train.txt' + '\n')
    f.write('val: ' + split_dir + fio.sep + 'val.txt' + '\n')
    f.write('test: ' + split_dir + fio.sep + 'test.txt' + '\n\n')
    f.write('nc: 1\n')
    f.write('names:\n')
    f.write('  0: chessboard')

### 2. Install YOLOv8 and Import

conda install pytorch torchvision torchaudio pytorch-cuda=12.1 ultralytics -c pytorch -c nvidia -c conda-forge

In [2]:
from ultralytics import YOLO
import time

### 3. Train YOLOv8

In [11]:
def current_timestamp(micro_second=False):
    t = time.time()
    if micro_second:
        return int(t * 1000 * 1000)
    else:
        return int(t * 1000)

In [13]:
model = YOLO("yolov8n.yaml").load("yolov8n.pt")

save_path = fio.createPath(fio.sep, [cwd, 'data', str(current_timestamp(False))])
fio.ensure_dir(save_path)
result = model.train(data=yaml_filepath, epochs=6, device=[], save_dir=save_path)

Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/chessboard.yaml, epochs=6, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[], workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False

train: Scanning /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels... 123 images, 0 backgrounds, 0 corrupt: 100%|██████████| 123/123 [00:00

train: New cache created: /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache



val: Scanning /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<00:0

val: New cache created: /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache


...

        6/6       2.1G     0.6127     0.9717      1.181         27        640: 100%|██████████| 8/8 [00:00<00:00, 26.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 30.17it/s]

                   all         15         15      0.992          1      0.995      0.916



6 epochs completed in 0.001 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 31.74it/s]


                   all         15         15      0.992          1      0.995      0.916
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train5


### 4. Visualise the training result

In [14]:
metrics = model.val()
metrics.box.map
metrics.box.map50
metrics.box.map75
metrics.box.maps

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


                   all         15         15      0.992          1      0.995      0.916
Speed: 0.1ms preprocess, 14.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train52


array([    0.91618])

### 5. Prediction

#### 1. Local testing set

In [7]:
predict_image_folder = fio.createPath(fio.sep, [cwd, 'data_predict', 'Chessboard_test1'])
predict_image_paths = fio.traverse_dir(predict_image_folder, full_path=True, towards_sub=False)
predict_image_paths = fio.filter_ext(filepath_list=predict_image_paths, filter_out_target=False, ext_set=fio.img_ext_set)

In [5]:
model_file_path = fio.createPath(fio.sep, [cwd, 'models', 'train5', 'weights'], 'best.pt')
model = YOLO(model_file_path)

In [8]:
predictions = model(predict_image_paths)
probs_save_dir = fio.createPath(fio.sep, [save_path], 'predictions')
fio.ensure_dir(probs_save_dir)

index = 0
for pred in predictions:
    box = pred.boxes
    masks = pred.masks
    keypoints = pred.keypoints
    probs = pred.probs

    original_image_path = predict_image_paths[index]
    original_image_name = fio.get_filename_components(original_image_path)[1]
    probs_save_path = fio.createPath(fio.sep, [save_path, 'predictions'], str(original_image_name) + '.jpg')
    pred.save(probs_save_path)
    index += 1


0: 640x640 (no detections), 2.0ms
1: 640x640 (no detections), 2.0ms
2: 640x640 (no detections), 2.0ms
3: 640x640 (no detections), 2.0ms
4: 640x640 (no detections), 2.0ms
5: 640x640 (no detections), 2.0ms
6: 640x640 1 chessboard, 2.0ms
7: 640x640 1 chessboard, 2.0ms
8: 640x640 1 chessboard, 2.0ms
9: 640x640 (no detections), 2.0ms
10: 640x640 (no detections), 2.0ms
11: 640x640 (no detections), 2.0ms
12: 640x640 (no detections), 2.0ms
13: 640x640 1 chessboard, 2.0ms
14: 640x640 1 chessboard, 2.0ms
15: 640x640 (no detections), 2.0ms
16: 640x640 (no detections), 2.0ms
17: 640x640 1 chessboard, 2.0ms
18: 640x640 (no detections), 2.0ms
19: 640x640 (no detections), 2.0ms
20: 640x640 1 chessboard, 2.0ms
21: 640x640 1 chessboard, 2.0ms
22: 640x640 1 chessboard, 2.0ms
23: 640x640 (no detections), 2.0ms
24: 640x640 (no detections), 2.0ms
25: 640x640 (no detections), 2.0ms
26: 640x640 1 chessboard, 2.0ms
27: 640x640 (no detections), 2.0ms
28: 640x640 (no detections), 2.0ms
29: 640x640 (no detectio

NameError: name 'save_path' is not defined

#### 2. Export YOLOv8 Custom Model to CoreML Package

In [38]:
model_file_path = fio.createPath(fio.sep, [cwd, 'models', 'train5', 'weights'], 'best.pt')
custom_yolo = YOLO(model_file_path)
custom_yolo.export(format='coreml')

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CPU (Intel Core(TM) i9-14900K)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients

PyTorch: starting from '/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/models/train5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

CoreML: starting export with coremltools 7.1...


Running MIL backend_mlprogram pipeline: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 723.29 passes/s]


CoreML: export success ✅ 1.8s, saved as '/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/models/train5/weights/best.mlpackage' (5.9 MB)

Export complete (3.1s)
Results saved to /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/models/train5/weights
Predict:         yolo predict task=detect model=/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/models/train5/weights/best.mlpackage imgsz=640  
Validate:        yolo val task=detect model=/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/models/train5/weights/best.mlpackage imgsz=640 data=/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/chessboard.yaml  
Visualize:       https://netron.app


'/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/models/train5/weights/best.mlpackage'